# From photo to piece


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from snap_fit.config.snap_fit_config import get_snap_fit_paths
from snap_fit.image.process import (
    find_contours,
    find_corners,
    find_sift_keypoints,
    find_white_regions,
)
from snap_fit.image.utils import (
    cut_rect_from_image,
    display_image,
    draw_contour,
    draw_contours,
    draw_corners,
    draw_keypoints,
    draw_regions,
    flip_colors_bw,
    load_image,
    pad_rect,
    save_image,
    show_image_mpl,
    sort_rects,
    translate_contour,
)
from snap_fit.puzzle.sheet import Sheet

In [ ]:
sf_paths = get_snap_fit_paths()
data_fol = sf_paths.data_fol
sample_fol = data_fol / "sample"

In [ ]:
# img_fn = "PXL_20241130_105107220.jpg"
# img_fn = "front_01.jpg"
# img_fn = "back_proc_02.jpg"
img_fn = "back_02.jpg"
# img_fn = "puzzle_pieces_01.jpeg"
img_fp = sample_fol / img_fn
img_fp

In [ ]:
sheet = Sheet(img_fp)

In [ ]:
show_image_mpl(sheet.img_orig)

In [ ]:
show_image_mpl(sheet.img_bw)

In [ ]:
for i, (x, y, w, h) in enumerate(sheet.regions[:10]):
    print(
        f"Black Region {i + 1}: x={x}, y={y}, width={w}, height={h}, area={w*h/1000:.2f}k"
    )

In [ ]:
annotated_image = draw_regions(sheet.img_orig, sheet.regions[:10])
show_image_mpl(annotated_image)

In [ ]:
for i, contour in enumerate(sheet.contours[:1]):
    print(f"Contour {i + 1}: {contour}")

In [ ]:
contour_image = draw_contours(sheet.img_orig, sheet.contours)

show_image_mpl(contour_image)

## grab a chunk of image


In [ ]:
region = sheet.regions[0]
region

In [ ]:
region_pad = pad_rect(region, 30, sheet.img_bw)
region_pad

In [ ]:
annotated_image = draw_regions(annotated_image, [region_pad])
show_image_mpl(annotated_image)

#### Use the Piece class


In [ ]:
piece = sheet.pieces[0]

In [ ]:
show_image_mpl(piece.img_bw)

In [ ]:
show_image_mpl(piece.img_orig)

### Find corners in the piece


In [ ]:
corners = find_corners(
    piece.img_bw,
    max_corners=10,
)
corners

In [ ]:
img_corners = draw_corners(piece.img_orig, corners)
show_image_mpl(img_corners)

#### Use SIFT and SURF


In [ ]:
# from snap_fit.image.process import find_surf_keypoints

In [ ]:
# keypoints, descriptors = find_surf_keypoints(piece.img_bw)
keypoints, descriptors = find_sift_keypoints(piece.img_bw)

In [ ]:
img_keypoints = draw_keypoints(piece.img_orig, keypoints)
show_image_mpl(img_keypoints)

## use contours


In [ ]:
piece = sheet.pieces[2]

In [ ]:
contour = piece.contour
contour

In [ ]:
x = -piece.region_pad[0]
y = -piece.region_pad[1]
print(f"Translating contour by {piece.region=} {x=}, {y=}")

c1 = translate_contour(contour, x, y)
c1

In [ ]:
# draw the contour on the image

img_contour = draw_contour(piece.img_orig, c1)
# img_contour = draw_contour(piece.img_bw, c1, color=127)
show_image_mpl(img_contour)

In [ ]:
# draw the contour on the image

img_contour = draw_contour(piece.img_orig, piece.contour_loc)
# img_contour = draw_contour(piece.img_bw, c1, color=127)
show_image_mpl(img_contour)

### Derivative of a contour


In [ ]:
piece = sheet.pieces[2]

In [ ]:
cl = piece.contour_loc

In [ ]:
cl.shape

In [ ]:
cl[:10, 0]

In [ ]:
cl[-10:, 0]

In [ ]:
from snap_fit.image.contour import Contour


cc = Contour(cl)
cc.derive(step=5)
print(cc.derivative.shape)
cc.derivative[:, 0]

In [ ]:
# cl[3][0]
# cc.derivative[3][0]

In [ ]:
from snap_fit.image.utils import draw_contour_derivative


cont_d_image = draw_contour_derivative(
    piece.img_orig,
    cl,
    cc.derivative,
    skip=3,
    arrow_length=20,
)
show_image_mpl(cont_d_image)

In [ ]:
# compute the orientation of the gradient

d = cc.derivative
d[:, 0]